In [1]:
from PIL import Image, ImageSequence
import numpy as np
import glob
import yaml
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, RobustScaler

In [2]:
def load_image(img_path, sample):
    # Open image and hunt down separate channels
    with Image.open(img_path) as im:
        channels = []
        for ch in ImageSequence.Iterator(im):
            ch = np.array(ch)
            channels.append(ch)
        
    channels = np.stack(channels, axis=2)
    dim = channels.shape
    channels_pca_rbg = PCA(n_components=3).fit_transform(channels.reshape(-1, dim[-1]))
    scaled_image = MinMaxScaler(feature_range=(0, 255)).fit_transform(channels_pca_rbg)
    scaled_image = scaled_image.reshape(*dim[:2], 3)
    
    #scaled_image = ((channels_pca_rbg - channels_pca_rbg.min(axis=(0,1), keepdims=True)) / (channels_pca_rbg.max(axis=(0,1), keepdims=True) - channels_pca_rbg.min(axis=(0,1), keepdims=True))) * 255
    scaled_image = scaled_image.astype(np.uint8)
    scaled_image = Image.fromarray(scaled_image)
    plt.imshow(scaled_image)
    plt.show()
    scaled_image.save(f"data/{sample}_pca_rgb.jpg")

In [3]:
with open("info.yaml", "r") as stream:
    INFO = yaml.safe_load(stream)
SAMPLES = INFO["SAMPLE"]
len(SAMPLES)

100